## Pymaceuticals, Inc.

#### Analysis

In [80]:
#Import Dependencies

import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np


In [81]:
#csv data files to load
mouse_metadata_file = "data/Mouse_metadata.csv"
study_results_file = "data/Study_results.csv"

#Read the mouse data and study results
mouse_metadata = pd.read_csv(mouse_metadata_file)
study_results = pd.read_csv(study_results_file)



In [82]:
#View mouse_metadata
mouse_metadata.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [83]:
#View study_results
study_results.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [84]:
#Merge data files together into a single data set on the common column "Mouse ID"
mouse_study_df = pd.merge(study_results, mouse_metadata, on="Mouse ID")
mouse_study_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22


In [85]:
#Find the number of unique mice in the data set by using unique
mouse = mouse_study_df["Mouse ID"].unique()
len(mouse)

249

In [86]:
#Find the number of mice in the data set by using value_counts
mice = mouse_study_df["Mouse ID"].value_counts()
len(mice)

249

In [87]:
#Check for any mouse ID with duplicate time points
duplicates = mouse_study_df.loc[mouse_study_df.duplicated(subset=["Mouse ID", "Timepoint"]), "Mouse ID"].unique()
duplicates

array(['g989'], dtype=object)

In [88]:
# Optional: Get all the data for the duplicate mouse ID
duplicate_df2 = mouse_study_df[mouse_study_df.duplicated(["Mouse ID", "Timepoint"])]
duplicate_df2

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
861,g989,0,45.000000,0,Propriva,Female,21,26
863,g989,5,47.570392,0,Propriva,Female,21,26
865,g989,10,49.880528,0,Propriva,Female,21,26
867,g989,15,53.442020,0,Propriva,Female,21,26
869,g989,20,54.657650,1,Propriva,Female,21,26


In [89]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID
clean_mouse_study_df = mouse_study_df[mouse_study_df["Mouse ID"]!= "g989"]
clean_mouse_study_df


,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,b128,5,45.651331,0,Capomulin,Female,9,22
2,b128,10,43.270852,0,Capomulin,Female,9,22
3,b128,15,43.784893,0,Capomulin,Female,9,22
4,b128,20,42.731552,0,Capomulin,Female,9,22
...,...,...,...,...,...,...,...,...
1888,m601,25,33.118756,1,Capomulin,Male,22,17
1889,m601,30,31.758275,1,Capomulin,Male,22,17
1890,m601,35,30.834357,1,Capomulin,Male,22,17
1891,m601,40,31.378045,1,Capomulin,Male,22,17


In [90]:
# Checking the number of mice in the clean DataFrame
len(clean_mouse_study_df["Mouse ID"].unique())

248

## Summary Statistics

In [97]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
clean_mouse_study_df["Drug Regimen"].unique()

array(['Capomulin', 'Ketapril', 'Naftisol', 'Infubinol', 'Stelasyn',
       'Ramicane', 'Zoniferol', 'Propriva', 'Placebo', 'Ceftamin'],
      dtype=object)

In [94]:
#Create a grouped dataframe by Drug Regimen
tumor_regimen_stats = clean_mouse_study_df.groupby("Drug Regimen")


In [99]:
#Find the mean for each drug regimen
mean = tumor_regimen_stats['Tumor Volume (mm3)'].mean()

#Find the median for each drug regimen
median = tumor_regimen_stats['Tumor Volume (mm3)'].median()

#Find the variance for each drug regimen
variance = tumor_regimen_stats['Tumor Volume (mm3)'].var()

#Find the standard deviation for each drug regimen
std = tumor_regimen_stats['Tumor Volume (mm3)'].std()

#Find the standard error of mean
sem = tumor_regimen_stats['Tumor Volume (mm3)'].sem()

#Combine all stats together into a dataframe to display results
summary_statistics = pd.DataFrame({
    "Mean Tumor Volume" : mean, 
    "Median Tumor Volume" : median, 
    "Tumor Volume Variance" : variance, 
    "Tumor Volume Std. Dev." : std, 
    "Tumor Volume Std. Err.": sem})

summary_statistics

,Mean Tumor Volume,Median Tumor Volume,Tumor Volume Variance,Tumor Volume Std. Dev.,Tumor Volume Std. Err.
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111
